In [ ]:
import datetime
def next_weekday(d, weekday):
    days_ahead = weekday - d.weekday()
    if days_ahead <= 0: # Target day already happened this week
        days_ahead += 7
    return d + datetime.timedelta(days_ahead)

d = datetime.datetime.today() 

if d.strftime('%A') == "Sunday":
    d = d - datetime.timedelta(1)
next_sunday = next_weekday(d, 6) # 0 = Monday, 1=Tuesday, 2=Wednesday...
week_str = []
week_str.append(str(next_sunday.year))
if len(str(next_sunday.month)) == 1:
    week_str.append('0' + str(next_sunday.month))
else:
    week_str.append(str(next_sunday.month))

if len(str(next_sunday.day)) == 1:
    week_str.append('0' + str(next_sunday.day))
else:
    week_str.append(str(next_sunday.day))
week_str = ''.join(week_str)
print(week_str)

In [ ]:
import pandas as pd


df = pd.read_csv("rawdata/raw_data_" + week_str + ".csv")
df['Time (h)'] = df['time_spent'] / 60
df['Distance (km)'] = df['dist_float']
df['Pace (km/min)'] = df['avg_pace_km_min']

In [ ]:
df.head()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

df_dist_per_person = df.copy(deep=True)
df_athletes_by_location = df.groupby(['location']).count().reset_index()
df_athletes_by_location['count'] = df_athletes_by_location['Rank']
df_athletes_by_location = df_athletes_by_location[['location', 'count']]

df_dist_per_person = df_dist_per_person.groupby(['location'])['Distance (km)'].sum().reset_index()
df_dist_per_person = df_dist_per_person.merge(df_athletes_by_location, on=['location'])
df_dist_per_person['Km per Athlete'] = round(df_dist_per_person['Distance (km)'] / df_dist_per_person['count'], 2)
df_dist_per_person = df_dist_per_person.sort_values(by=['Km per Athlete'], ascending=False)
df_dist_per_person['Rank'] = [x for x in range(1, len(df_dist_per_person) + 1)]

df_dist_per_person = df_dist_per_person[['Rank', 'Km per Athlete', 'location']]

fig_table_dist_per_person = go.Figure(data=[go.Table(
    header=dict(values=df_dist_per_person.columns,
                fill_color='paleturquoise',
                align='center'),
    cells=dict(values=[df_dist_per_person['Rank'], df_dist_per_person['Km per Athlete'], df_dist_per_person['location']],
               fill_color='lavender',
               align='center'))
])

fig_table_dist_per_person.update_layout(shapes=[go.layout.Shape(
type='rect',
xref='paper',
yref='paper',
x0=0,
y0=-0.1,
x1=1.01,
y1=1.02,
line={'width': 1, 'color': 'black'}
)])

fig_table_dist_per_person.update_traces(cells_font=dict(size = 15))

fig_table_dist_per_person.update_layout(title_text='National Rank by Distance per Athlete for Week Ending ' + week_str, title_x=0.5, width=900, height=410)
fig_table_dist_per_person.show()

In [ ]:
import plotly.express as px

fig_dist_time = px.scatter(df, x="Distance (km)", y="Time (h)", color="location",
                 width=200, height=200)
fig_dist_time.update_layout(title_text='Total Distance vs Total Time for Week Ending ' + week_str, title_x=0.5)
fig_dist_time.update_layout(shapes=[go.layout.Shape(
type='rect',
xref='paper',
yref='paper',
x0=0,
y0=-0.1,
x1=1.01,
y1=1.02,
line={'width': 1, 'color': 'black'}
)])
fig_dist_time.show()
# fig_dist_time.write_html("dist_v_time.html")

In [ ]:
import plotly.express as px

fig_dist_pace = px.scatter(df, x="Distance (km)", y="Pace (km/min)", color="location", width=200, height=200)
fig_dist_pace.update_layout(title_text='Total Distance vs Pace for Week Ending ' + week_str, title_x=0.5)

fig_dist_pace.update_layout(shapes=[go.layout.Shape(
type='rect',
xref='paper',
yref='paper',
x0=0,
y0=-0.1,
x1=1.01,
y1=1.02,
line={'width': 1, 'color': 'black'}
)])
fig_dist_pace.show()
# fig_dist_pace.write_html("dist_v_pace.html")

In [ ]:
import plotly.express as px


df_tot_dist_loc = df.copy(deep=True)
df_tot_dist_loc = df_tot_dist_loc.groupby(['location']).sum('Distance (km)').reset_index()
df_tot_dist_loc = df_tot_dist_loc.sort_values(by=['Distance (km)'], ascending=True)
fig_bar = px.bar(df_tot_dist_loc, x='Distance (km)', y='location', hover_data=['location'], width=200, height=200, orientation='h')
fig_bar.update_layout(title_text='Total Distance by Location for Week Ending ' + week_str, title_x=0.5)

fig_bar.update_layout(shapes=[go.layout.Shape(
type='rect',
xref='paper',
yref='paper',
x0=0,
y0=-0.1,
x1=1.01,
y1=1.02,
line={'width': 1, 'color': 'black'}
)])

fig_bar.show()
# fig_bar.write_html("dist_loca_bar.html")

In [ ]:
import plotly.express as px


df_avg_pace_loc = df.copy(deep=True)
df_avg_pace_loc = df_avg_pace_loc.groupby(['location']).mean('Pace (km/min)').reset_index()
df_avg_pace_loc = df_avg_pace_loc.sort_values(by=['Pace (km/min)'], ascending=False)
fig_bar_avg_pace = px.bar(df_avg_pace_loc, x='Pace (km/min)', y='location', hover_data=['location'], width=200, height=200, orientation='h')
fig_bar_avg_pace.update_layout(title_text='Average Pace by Location for Week Ending ' + week_str, title_x=0.5)


fig_bar_avg_pace.update_layout(shapes=[go.layout.Shape(
type='rect',
xref='paper',
yref='paper',
x0=0,
y0=-0.1,
x1=1.01,
y1=1.02,
line={'width': 1, 'color': 'black'}
)])
fig_bar_avg_pace.show()
# fig_bar_avg_pace.write_html("dist_pace_loc_bar.html")

In [ ]:
import plotly.graph_objects as go
import pandas as pd

df_dist_rank = df[['Athlete', 'Distance (km)', 'location']].copy(deep=True)
df_dist_rank = df_dist_rank.sort_values(by=['Distance (km)'], ascending=False)

df_dist_rank['Rank'] = [x for x in range(1, len(df_dist_rank) + 1)]
df_dist_rank = df_dist_rank[['Rank', 'Athlete', 'Distance (km)', 'location']]

fig_table = go.Figure(data=[go.Table(
    header=dict(values=df_dist_rank.columns,
                fill_color='paleturquoise',
                align='center'),
    cells=dict(values=[df_dist_rank['Rank'], df_dist_rank['Athlete'], df_dist_rank['Distance (km)'], df_dist_rank['location']],
               fill_color='lavender',
               align='center'))
])

fig_table.update_traces(cells_font=dict(size = 15))

fig_table.update_layout(title_text='National Rank by Total Distance for Week Ending ' + week_str, title_x=0.5, width=900, height=500)
fig_table.update_layout(shapes=[go.layout.Shape(
type='rect',
xref='paper',
yref='paper',
x0=0,
y0=-0.1,
x1=1.01,
y1=1.02,
line={'width': 1, 'color': 'black'}
)])
fig_table.show()
# fig_table.write_html("dist_rank_table.html")

In [ ]:
import plotly.graph_objects as go
import pandas as pd

df_dist_rank_clubs = df[['Distance (km)', 'location']].copy(deep=True)
df_dist_rank_clubs = df_dist_rank_clubs.groupby(['location']).sum('Distance (km)').reset_index()
df_dist_rank_clubs['Distance (km)'] = round(df_dist_rank_clubs['Distance (km)'], 2)
df_dist_rank_clubs = df_dist_rank_clubs.sort_values(by=['Distance (km)'], ascending=False)

df_dist_rank_clubs['Rank'] = [x for x in range(1, len(df_dist_rank_clubs) + 1)]
df_dist_rank_clubs = df_dist_rank_clubs[['Rank', 'Distance (km)', 'location']]

fig_table_clubs = go.Figure(data=[go.Table(
    header=dict(values=df_dist_rank_clubs.columns,
                fill_color='paleturquoise',
                align='center'),
    cells=dict(values=[df_dist_rank_clubs['Rank'], df_dist_rank_clubs['Distance (km)'], df_dist_rank_clubs['location']],
               fill_color='lavender',
               align='center'))
])

fig_table_clubs.update_traces(cells_font=dict(size = 15))

fig_table_clubs.update_layout(title_text='National Rank by Total Distance for Week Ending ' + week_str, title_x=0.5, width=900, height=410)

fig_table_clubs.update_layout(shapes=[go.layout.Shape(
type='rect',
xref='paper',
yref='paper',
x0=0,
y0=-0.1,
x1=1.01,
y1=1.02,
line={'width': 1, 'color': 'black'}
)])
fig_table_clubs.show()
# fig_table_clubs.write_html("dist_rank_table_clubs.html")

In [ ]:
import plotly.graph_objects as go
import pandas as pd

df_pace_rank_clubs = df[['Pace (km/min)', 'location']].copy(deep=True)
df_pace_rank_clubs = df_pace_rank_clubs.groupby(['location']).mean('Pace (km/min)').reset_index()
df_pace_rank_clubs['Pace (km/min)'] = df_pace_rank_clubs['Pace (km/min)'].map(lambda x: round(x, 3))
df_pace_rank_clubs = df_pace_rank_clubs.sort_values(by=['Pace (km/min)'], ascending=True)

df_pace_rank_clubs['Rank'] = [x for x in range(1, len(df_pace_rank_clubs) + 1)]
df_pace_rank_clubs = df_pace_rank_clubs[['Rank', 'Pace (km/min)', 'location']]

fig_table_pace_clubs = go.Figure(data=[go.Table(
    header=dict(values=df_pace_rank_clubs.columns,
                fill_color='paleturquoise',
                align='center'),
    cells=dict(values=[df_pace_rank_clubs['Rank'], df_pace_rank_clubs['Pace (km/min)'], df_pace_rank_clubs['location']],
               fill_color='lavender',
               align='center'))
])

fig_table_pace_clubs.update_traces(cells_font=dict(size = 15))

fig_table_pace_clubs.update_layout(title_text='National Rank by Average Pace (km/min) for Week Ending ' + week_str, title_x=0.5, width=900, height=410)
fig_table_pace_clubs.update_layout(shapes=[go.layout.Shape(
type='rect',
xref='paper',
yref='paper',
x0=0,
y0=-0.1,
x1=1.01,
y1=1.02,
line={'width': 1, 'color': 'black'}
)])
fig_table_pace_clubs.show()
# fig_table_pace_clubs.write_html("pace_rank_table_clubs.html")

In [ ]:
import plotly.express as px
from plotly.offline import plot
from plotly.subplots import make_subplots

figures = [
            fig_bar,
            fig_bar_avg_pace,
    ]

fig = make_subplots(rows=len(figures), cols=1, subplot_titles=('Total Distance by Location for Week Ending ' + week_str, "Average Pace by Location for Week Ending " + week_str)) 

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        fig.append_trace(figure["data"][trace], row=i+1, col=1)
        

In [ ]:
import plotly.express as px
from plotly.offline import plot
from plotly.subplots import make_subplots

figures_scatter = [
            fig_dist_time,
            fig_dist_pace,
    ]

fig_scatter = make_subplots(rows=len(figures_scatter), cols=1, subplot_titles=("Total Distance vs Total Time for Week Ending " + week_str, "Total Distance vs Pace for Week Ending " + week_str)) 

for i, figure in enumerate(figures_scatter):
    for trace in range(len(figure["data"])):
        fig_scatter.append_trace(figure["data"][trace], row=i+1, col=1)
        

# Progress

In [ ]:
import pandas as pd
import glob
import os


path = 'rawdata' # use your path
all_files = os.listdir('rawdata')
li = []

for filename in all_files:
    df_raws = pd.read_csv('rawdata/' + filename, index_col=None, header=0)
    _week = int(filename.split('.')[0].split('_')[-1])
    df_raws['week'] = _week
    li.append(df_raws)


df_all_raws = pd.concat(li, axis=0, ignore_index=True)
df_all_raws['Distance (km)'] = df_all_raws['dist_float']
df_all_raws_wbw = df_all_raws.groupby(['week', 'location']).sum(['Distance (km)']).reset_index()
df_all_raws_wbw = df_all_raws_wbw.sort_values(by=['week'])
df_all_raws_wbw['week'] = df_all_raws_wbw['week'].map(lambda x: str(x))

fig_dist_weekly = px.line(df_all_raws_wbw, x="week", y="Distance (km)", color="location", width=900, height=350)
fig_dist_weekly.update_layout(title_text='Weekly Distance (km) by Club', title_x=0.5)
fig_dist_weekly.update_layout(shapes=[go.layout.Shape(
type='rect',
xref='paper',
yref='paper',
x0=0,
y0=-0.1,
x1=1.01,
y1=1.02,
line={'width': 1, 'color': 'black'}
)])
fig_dist_weekly.show()

In [ ]:
import pandas as pd
import glob
import os


path = 'rawdata' # use your path
all_files = os.listdir('rawdata')
li = []

for filename in all_files:
    df_raws = pd.read_csv('rawdata/' + filename, index_col=None, header=0)
    _week = int(filename.split('.')[0].split('_')[-1])
    df_raws['week'] = _week
    li.append(df_raws)


df_all_raws = pd.concat(li, axis=0, ignore_index=True)
df_all_raws['Distance (km)'] = df_all_raws['dist_float']
df_all_raws_wbw = df_all_raws.groupby(['week', 'location']).sum(['Distance (km)']).reset_index()
df_all_raws_wbw = df_all_raws_wbw.sort_values(by=['week', 'Distance (km)'], ascending=False)
df_all_raws_wbw['week'] = df_all_raws_wbw['week'].map(lambda x: str(x))


fig_grouped_bar_weekly = px.bar(df_all_raws_wbw, x='Distance (km)', y='week', color='location', width=900, height=500, orientation='h', barmode="group")
fig_grouped_bar_weekly.update_layout(title_text='Weekly Distance (km) by Club', title_x=0.5, barmode='group', bargap=0.30, bargroupgap=0.0)

# fig_grouped_bar_weekly.update_layout(shapes=[go.layout.Shape(
# type='rect',
# xref='paper',
# yref='paper',
# x0=0,
# y0=-0.1,
# x1=1.01,
# y1=1.02,
# line={'width': 1, 'color': 'black'}
# )])
fig_grouped_bar_weekly.show()

In [ ]:
import os
import base64
data_uri = base64.b64encode(open('plots_by_week/nmrcall.png', 'rb').read()).decode('utf-8')
img_tag = '<img src="data:image/png;base64,{0}">'.format(data_uri)


try:
    os.remove("plots_by_week/MRC/index.html")
except:
    pass

try:
    os.remove("plots_by_week/index.html")
except:
    pass

try:
    os.remove('plots_by_week/mrc_all_plots_week_' + week_str + '.html')
except:
    pass

with open('plots_by_week/mrc_all_plots_week_' + week_str + '.html', 'a') as f:
    f.write(img_tag)
    f.write(fig_grouped_bar_weekly.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig_table_clubs.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig_table_dist_per_person.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig_table_pace_clubs.to_html(full_html=False, include_plotlyjs='cdn'))
#     f.write(fig_table.to_html(full_html=False, include_plotlyjs='cdn'))
#     f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
#     f.write(fig_scatter.to_html(full_html=False, include_plotlyjs='cdn'))
with open('plots_by_week/index.html', 'a') as f:
    f.write(img_tag)
    f.write(fig_grouped_bar_weekly.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig_table_clubs.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig_table_dist_per_person.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig_table_pace_clubs.to_html(full_html=False, include_plotlyjs='cdn'))
#     f.write(fig_table.to_html(full_html=False, include_plotlyjs='cdn'))
#     f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
#     f.write(fig_scatter.to_html(full_html=False, include_plotlyjs='cdn'))

with open('plots_by_week/MRC/index.html', 'a') as f:
    f.write(img_tag)
    f.write(fig_grouped_bar_weekly.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig_table_clubs.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig_table_dist_per_person.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig_table_pace_clubs.to_html(full_html=False, include_plotlyjs='cdn'))
#     f.write(fig_table.to_html(full_html=False, include_plotlyjs='cdn'))
#     f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
#     f.write(fig_scatter.to_html(full_html=False, include_plotlyjs='cdn'))